In [1]:
import pandas as pd
import json

# === Step 1: Load the original CSV ===

csv_path = "analytics_sample data-48-1.csv"

try:
    df = pd.read_csv(csv_path)
    print(f"✅ CSV loaded successfully with {len(df)} rows and {len(df.columns)} columns.")
except FileNotFoundError:
    print("❌ CSV file not found. Check the file path.")
    exit()

✅ CSV loaded successfully with 20985 rows and 7 columns.


In [2]:
df

,id,user_id,event,metadata,device,service,crt_dt
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
20980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20981,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20982,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20983,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# === Step 2: Remove unnecessary columns ===

columns_to_drop = ["id", "device", "service", "crt_dt"]
df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True, errors='ignore')
print(f"🧹 Removed columns: {columns_to_drop}")

🧹 Removed columns: ['id', 'device', 'service', 'crt_dt']


In [4]:
# === Step 3: Keep only necessary event types ===

allowed_events = [
    "post_commented",
    "post_liked",
    "post_viewed",
    "post_viewed_time"
]
df = df[df["event"].isin(allowed_events)].reset_index(drop=True)
print(f"📊 Filtered events: {allowed_events}")
print(f"✅ Remaining rows after filtering: {len(df)}")

📊 Filtered events: ['post_commented', 'post_liked', 'post_unliked', 'post_viewed', 'post_viewed_time']
✅ Remaining rows after filtering: 28


In [5]:
# === Step 4: Parse metadata JSON and extract post_id ===

def extract_post_id(metadata_str):
    try:
        metadata = json.loads(metadata_str)
        return metadata.get("post_id", None)
    except (json.JSONDecodeError, TypeError):
        return None

df["post_id"] = df["metadata"].apply(extract_post_id)
df.dropna(subset=["post_id"], inplace=True)
df["post_id"] = df["post_id"].astype(int, errors="ignore")

In [6]:
# === Step 5: Add 'view' count for post_viewed events ===

df["view"] = 0
view_df = df[df["event"] == "post_viewed"]
view_counts = view_df.groupby(["user_id", "post_id"]).size().reset_index(name="view")
df = df.merge(view_counts, on=["user_id", "post_id"], how="left", suffixes=("", "_count"))
df["view"] = df.apply(lambda x: x["view_count"] if x["event"] == "post_viewed" else 0, axis=1)
df.drop(columns=["view_count"], inplace=True, errors="ignore")

In [9]:
view_df.head(10)

,user_id,event,metadata,post_id,view
1,1.000003e+11,post_viewed,"{""mute"": ""N"", ""likes"": ""0"", ""saved"": ""N"", ""ban...",3151,0
2,1.000003e+11,post_viewed,"{""mute"": ""N"", ""likes"": ""0"", ""saved"": ""N"", ""ban...",3146,0
3,1.000003e+11,post_viewed,"{""mute"": ""N"", ""likes"": ""0"", ""saved"": ""N"", ""ban...",3151,0
5,1.000003e+11,post_viewed,"{""mute"": ""N"", ""likes"": ""0"", ""saved"": ""N"", ""ban...",3146,0
6,1.000003e+11,post_viewed,"{""mute"": ""N"", ""likes"": ""0"", ""saved"": ""N"", ""ban...",3078,0
7,1.000003e+11,post_viewed,"{""mute"": ""N"", ""likes"": ""0"", ""saved"": ""N"", ""ban...",3138,0
8,1.000003e+11,post_viewed,"{""mute"": ""N"", ""likes"": ""0"", ""saved"": ""N"", ""ban...",3138,0
9,1.011400e+04,post_viewed,"{""mute"": ""N"", ""likes"": ""0"", ""saved"": ""N"", ""ban...",3192,0
10,1.011400e+04,post_viewed,"{""mute"": ""N"", ""likes"": ""0"", ""saved"": ""N"", ""ban...",2891,0
11,1.011400e+04,post_viewed,"{""mute"": ""N"", ""likes"": ""0"", ""saved"": ""N"", ""ban...",3108,0


In [8]:
# === Step 6: Add 'like' count for post_liked events ===

df["like"] = 0
like_df = df[df["event"] == "post_liked"]
like_counts = like_df.groupby(["user_id", "post_id"]).size().reset_index(name="like")
df = df.merge(like_counts, on=["user_id", "post_id"], how="left", suffixes=("", "_count"))
df["like"] = df.apply(lambda x: x["like_count"] if x["event"] == "post_liked" else 0, axis=1)
df.drop(columns=["like_count"], inplace=True, errors="ignore")

In [10]:
# # === Step 7: Add 'unlike' count for post_unliked events (as negative) ===

# df["unlike"] = 0
# unlike_df = df[df["event"] == "post_unliked"]
# unlike_counts = unlike_df.groupby(["user_id", "post_id"]).size().reset_index(name="unlike")
# unlike_counts["unlike"] = -unlike_counts["unlike"]
# df = df.merge(unlike_counts, on=["user_id", "post_id"], how="left", suffixes=("", "_count"))
# df["unlike"] = df.apply(lambda x: x["unlike_count"] if x["event"] == "post_unliked" else 0, axis=1)
# df.drop(columns=["unlike_count"], inplace=True, errors="ignore")

In [ ]:
# # === Step 8: Remove rows where unlike != 0 ===

# initial_rows = len(df)
# df = df[df["unlike"] == 0].reset_index(drop=True)
# removed_rows = initial_rows - len(df)
# print(f"\n🧹 Removed {removed_rows} rows where unlike != 0")

In [ ]:
# # === Step 9: Add 'comment' text for post_commented events ===
# def extract_comment(metadata_str):
#     try:
#         metadata = json.loads(metadata_str)
#         for key in ["comment", "text", "message", "content", "body", "desc"]:
#             if key in metadata:
#                 return str(metadata[key]).strip()
#         return ""
#     except (json.JSONDecodeError, TypeError):
#         return ""

# # Extract comments for post_commented events
# comment_df = df[df["event"] == "post_commented"].copy()
# comment_df["comment"] = comment_df["metadata"].apply(extract_comment)

# # Combine multiple comments for same (user_id, post_id)
# comment_grouped = (
#     comment_df.groupby(["user_id", "post_id"])["comment"]
#     .apply(lambda x: " | ".join(filter(None, x)))
#     .reset_index()
# )

# # Merge back into main df
# df = df.merge(comment_grouped, on=["user_id", "post_id"], how="left", suffixes=("", "_text"))

# # Safely fill missing values (handles both column name cases)
# if "comment_text" in df.columns:
#     df["comment"] = df["comment_text"].fillna(df.get("comment", ""))
#     df.drop(columns=["comment_text"], inplace=True, errors="ignore")
# else:
#     df["comment"] = df["comment"].fillna("")

In [11]:
# === Step 9: Add 'comment' count for post_commented events ===

def has_comment_key(metadata_str):
    """Return True if metadata JSON has a 'comment' key."""
    try:
        metadata = json.loads(metadata_str)
        return "comment" in metadata and bool(str(metadata["comment"]).strip())
    except (json.JSONDecodeError, TypeError):
        return False

# Create a 'comment' column with 0 as default
df["comment"] = 0

# Filter only post_commented events that actually have a 'comment' key
comment_df = df[
    (df["event"] == "post_commented") & df["metadata"].apply(has_comment_key)
]

# Group and count number of comments per user_id + post_id
comment_counts = (
    comment_df.groupby(["user_id", "post_id"]).size().reset_index(name="comment")
)

# Merge counts back into main df
df = df.merge(comment_counts, on=["user_id", "post_id"], how="left", suffixes=("", "_count"))

# Assign comment count only for post_commented rows, else 0
df["comment"] = df.apply(lambda x: x["comment_count"] if x["event"] == "post_commented" else 0, axis=1)

# Drop helper column
df.drop(columns=["comment_count"], inplace=True, errors="ignore")


In [13]:
# === Step 10: Add 'view_time_sec' for post_viewed_time events ===

def extract_view_time(metadata_str):
    try:
        metadata = json.loads(metadata_str)
        return float(metadata.get("view_time", metadata.get("duration", 0)))
    except (json.JSONDecodeError, TypeError, ValueError):
        return 0.0

df["view_time_sec"] = 0.0
view_time_df = df[df["event"] == "post_viewed_time"].copy()
view_time_df["view_time_sec"] = view_time_df["metadata"].apply(extract_view_time)

view_time_grouped = (
    view_time_df.groupby(["user_id", "post_id"])["view_time_sec"]
    .sum()
    .reset_index()
)

df = df.merge(view_time_grouped, on=["user_id", "post_id"], how="left", suffixes=("", "_sum"))
df["view_time_sec"] = df.apply(
    lambda x: x["view_time_sec_sum"] if x["event"] == "post_viewed_time" else 0.0,
    axis=1,
)
df.drop(columns=["view_time_sec_sum"], inplace=True, errors="ignore")

In [14]:
# # Ensure 'comment' column is string type
# df["comment"] = df["comment"].fillna("").astype(str)

# # Aggregate metrics per (user_id, post_id)
# aggregated = (
#     df.groupby(["user_id", "post_id"], as_index=False)
#     .agg({
#         "view": "sum",
#         "like": "sum",
#         "view_time_sec": "sum",
#         "comment": lambda x: " | ".join(
#             str(i).strip() for i in x if pd.notna(i) and str(i).strip() != ""
#         ),
#     })
# )

In [15]:
# === Step: Aggregate metrics per (user_id, post_id) ===

# Ensure numeric columns have no NaNs
df["view"] = df["view"].fillna(0).astype(int)
df["like"] = df["like"].fillna(0).astype(int)
df["comment"] = df["comment"].fillna(0).astype(int)
df["view_time_sec"] = df["view_time_sec"].fillna(0).astype(float)

# Aggregate numeric metrics per user-video pair
aggregated = (
    df.groupby(["user_id", "post_id"], as_index=False)
    .agg({
        "view": "sum",
        "like": "sum",
        "comment": "sum",          # ✅ now summed instead of joined
        "view_time_sec": "sum",
    })
)

In [16]:
# # === Step 12: Optional cleanup - remove rows without engagement ===

# # Make sure comment column is a string (avoid .str errors)
# aggregated["comment"] = aggregated["comment"].fillna("").astype(str)

# # Keep only rows where user actually engaged
# aggregated = aggregated[
#     (aggregated["view"] > 0)
#     | (aggregated["like"] > 0)
#     | (aggregated["view_time_sec"] > 0)
#     | (aggregated["comment"].str.strip() != "")
# ].reset_index(drop=True)

# # Reorder and sort columns (optional, but nice for clarity)
# aggregated = aggregated[["user_id", "post_id", "view", "like", "view_time_sec", "comment"]]
# aggregated = aggregated.sort_values(by=["user_id", "post_id"]).reset_index(drop=True)

# # === Step 13: Save final aggregated dataset ===
# output_path = "aggregated_user_post_engagement.csv"
# aggregated.to_csv(output_path, index=False, encoding="utf-8")

# print("\n✅ Final dataset prepared for recommendation modeling!")
# print(f"📊 Total unique user-post pairs: {len(aggregated)}")
# print("🧠 Columns:", list(aggregated.columns))
# print("\nSample data:")
# print(aggregated.head(10))
# print(f"\n💾 Saved as: {output_path}")


✅ Final dataset prepared for recommendation modeling!
📊 Total unique user-post pairs: 19
🧠 Columns: ['user_id', 'post_id', 'view', 'like', 'view_time_sec', 'comment']

Sample data:
        user_id  post_id  view  like  view_time_sec comment
0  1.011400e+04     2891     4     0            0.0       0
1  1.011400e+04     2971     1     0            0.0       0
2  1.011400e+04     2987     1     0            0.0       0
3  1.011400e+04     3033     1     0            0.0       0
4  1.011400e+04     3040     1     0            0.0       0
5  1.011400e+04     3108     9     0            0.0       0
6  1.011400e+04     3192     4     0            0.0       0
7  1.011400e+04     3201     1     0            0.0       0
8  1.000003e+11     3138     4     0            0.0       0
9  1.000003e+11     1122     0     0            0.0       0

💾 Saved as: aggregated_user_post_engagement.csv


In [17]:
# === Step 12: Optional cleanup - remove rows without engagement ===

# Ensure numeric columns have no NaNs
aggregated["view"] = aggregated["view"].fillna(0).astype(int)
aggregated["like"] = aggregated["like"].fillna(0).astype(int)
aggregated["comment"] = aggregated["comment"].fillna(0).astype(int)
aggregated["view_time_sec"] = aggregated["view_time_sec"].fillna(0).astype(float)

# Keep only rows where user actually engaged (any metric > 0)
aggregated = aggregated[
    (aggregated["view"] > 0)
    | (aggregated["like"] > 0)
    | (aggregated["view_time_sec"] > 0)
    | (aggregated["comment"] > 0)
].reset_index(drop=True)

# Reorder and sort columns for clarity
aggregated = aggregated[["user_id", "post_id", "view", "like", "comment", "view_time_sec"]]
aggregated = aggregated.sort_values(by=["user_id", "post_id"]).reset_index(drop=True)

# === Step 13: Save final aggregated dataset ===
output_path = "aggregated_user_post_engagement.csv"
aggregated.to_csv(output_path, index=False, encoding="utf-8")

print("\n✅ Final dataset prepared for recommendation modeling!")
print(f"📊 Total unique user–post pairs: {len(aggregated)}")
print("🧠 Columns:", list(aggregated.columns))
print("\nSample data:")
print(aggregated.head(10))
print(f"\n💾 Saved as: {output_path}")


✅ Final dataset prepared for recommendation modeling!
📊 Total unique user–post pairs: 13
🧠 Columns: ['user_id', 'post_id', 'view', 'like', 'comment', 'view_time_sec']

Sample data:
        user_id  post_id  view  like  comment  view_time_sec
0  1.011400e+04     2891     4     0        0            0.0
1  1.011400e+04     2971     1     0        0            0.0
2  1.011400e+04     2987     1     0        0            0.0
3  1.011400e+04     3033     1     0        0            0.0
4  1.011400e+04     3040     1     0        0            0.0
5  1.011400e+04     3108     9     0        0            0.0
6  1.011400e+04     3192     4     0        0            0.0
7  1.011400e+04     3201     1     0        0            0.0
8  1.000003e+11     3138     4     0        0            0.0
9  1.000003e+11     3078     1     0        0            0.0

💾 Saved as: aggregated_user_post_engagement.csv


In [ ]:
final_df = pd.read_csv("aggregated_user_post_engagement.csv")

In [ ]:
final_df.head(50)

In [ ]:
final_df